In [ ]:

        # Determine the unloading capacity of each berth
        berth_service_rate = []
        for i in range (len(berths)):
            service_rate = []
            for j in range(4):
                for k in range(len(cranes[j])):
                    if cranes[j][k].berth == i+1:
                        service_rate.append(cranes[j][k].effective_capacity)                  
            berth_service_rate.append(int(np.sum(service_rate)))
        total_service_rate = np.sum(berth_service_rate)
        
        # Traffic distribution according to ratio service rate of each berth
        traffic_ratio = []
        for i in range (len(berths)):
            traffic_ratio.append(berth_service_rate[i]/total_service_rate)
            
        # Determine total time that vessels are at each berth
        occupancy = []
        service_times = []
        for i in range (len(berths)):            
            berth_time = []
            for j in range (3):
                calls          = vessels[j].calls[timestep] * traffic_ratio[i]
                service_time   = vessels[j].call_size / berth_service_rate[i]
                mooring_time   = vessels[j].mooring_time
                time_at_berth  = service_time + mooring_time
                berth_time.append(time_at_berth * calls)
            berth_time = np.sum(berth_time)
            service_times.append(service_time)
            occupancy.append(berth_time / operational_hours)
        
        # Determine the waiting time of each vessel type
        waiting_times = []
        waiting_factors = []
        for i in range (len(berths)):    
            factor = berths[0].occupancy_to_waitingfactor(occupancy[i], len(berths))
            vessel_wait_times = []
            for j in range (3):
                calls           = vessels[j].calls[timestep] * traffic_ratio[i]
                service_time    = vessels[j].call_size / berth_service_rate[i]
                individual_wait = service_time * factor
                vessel_wait_times.append(individual_wait)
            waiting_times.append(max(vessel_wait_times))
            waiting_factors.append(factor)